# ***Pkg Installation & Drive Mount***

In [ ]:
from google.colab import drive
import sys
import os

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git
drive.mount('/content/drive', force_remount=True)
sys.path.append('path_to_your_code')

# ***Datasets Generation***

In [ ]:
from scipy.linalg import expm
import torch
import csv
import os
import numpy as np
import random
import sklearn.preprocessing as skp
from torch.utils.data import Dataset
import pandas as pd
from datetime import datetime
from typing import List, Tuple
from functools import lru_cache
from tqdm import tqdm
import math


class MyDataset(Dataset):
    def __init__(self, root: str, desti: str, market: str, comlist: List[str], start: str, end: str, window: int, dataset_type: str):
        super().__init()

        self.comlist = comlist
        self.market = market
        self.root = root
        self.desti = desti
        self.start = start
        self.end = end
        self.window = window
        self.dates, self.next_day = self.find_dates(self.start, self.end, self.root, self.comlist, self.market)
        self.dataset_type = dataset_type

        # Check if graph files already exist
        graph_files_exist = all(os.path.exists(os.path.join(self.desti, f'{self.market}_{self.dataset_type}_{self.start}_{self.end}_{self.window}/graph_{i}.pt')) for i in range(len(self.dates) - self.window + 1))

        if not graph_files_exist:
            # Generate the graphs and save them as PyTorch tensors
            self._create_graphs(self.dates, self.desti, self.comlist, self.market, self.root, self.window, self.next_day)

    def __len__(self):
        return len(self.dates) - self.window + 1

    def __getitem__(self, idx: int):
        directory_path = os.path.join(self.desti, f'{self.market}_{self.dataset_type}_{self.start}_{self.end}_{self.window}')
        data_path = os.path.join(directory_path, f'graph_{idx}.pt')

        if os.path.exists(data_path):
            return torch.load(data_path)
        else:
            raise FileNotFoundError(f"No graph data found for index {idx}. Please ensure you've generated the required data.")

    def check_years(self, date_str: str, start_str: str, end_str: str) -> bool:
        date_format = "%Y-%m-%d"
        date = datetime.strptime(date_str, date_format)
        start = datetime.strptime(start_str, date_format)
        end = datetime.strptime(end_str, date_format)

        return start <= date <= end

    def find_dates(self, start: str, end: str, path: str, comlist: List[str], market: str) -> Tuple[List[str], str]:
        # Get the dates for each company in the target list
        date_sets = []
        after_end_date_sets = []
        for h in comlist:
            dates = set()
            after_end_dates = set()
            d_path = os.path.join(path, f'{market}_{h}_30Y.csv')
            with open(d_path, 'r') as f:
                file = csv.reader(f)
                next(file, None)  # Skip the header row
                for line in file:
                    date_str = line[0][:10]
                    if self.check_years(date_str, start, end):
                        dates.add(date_str)
                    elif self.check_years(date_str, end, '2017-12-31'): # '2017-12-31' is just an example, if the latest data is used, fill in the current date
                        after_end_dates.add(date_str)

            date_sets.append(dates)
            after_end_date_sets.append(after_end_dates)

        # Find the intersection of all date sets
        all_dates = list(set.intersection(*date_sets))
        all_after_end_dates = list(set.intersection(*after_end_date_sets))

        # Find the next common day after the end date
        next_common_day = min(all_after_end_dates) if all_after_end_dates else None

        return sorted(all_dates), next_common_day

    def signal_energy(self, x_tuple: Tuple[float]) -> float:
        x = np.array(x_tuple)
        return np.sum(np.square(x))

    def information_entropy(self, x_tuple: Tuple[float]) -> float:
        x = np.array(x_tuple)
        unique, counts = np.unique(x, return_counts=True)
        probabilities = counts / np.sum(counts)
        entropy = -np.sum(probabilities * np.log(probabilities))

        return entropy

    def adjacency_matrix(self, X: torch.Tensor) -> torch.Tensor:
        A = torch.zeros((X.shape[0], X.shape[0]))
        X = X.numpy()
        energy = np.array([self.signal_energy(tuple(x)) for x in X])
        entropy = np.array([self.information_entropy(tuple(x)) for x in X])
        for i in range(X.shape[0]):
            for j in range(X.shape[0]):
                A[i, j] = torch.tensor((energy[i] / energy[j]) * (math.exp(entropy[i] - entropy[j])), dtype=torch.float32)

        return torch.log(A[A<1] = 1)

    def node_feature_matrix(self, dates: List[str], comlist: List[str], market: str, path: str) -> torch.Tensor:
        # Convert dates to datetime format for easier comparison
        dates_dt = [pd.to_datetime(date).date() for date in dates]

        # Initialize the tensor
        X = torch.zeros((5, len(comlist), len(dates_dt)))

        for idx, h in enumerate(comlist):
            d_path = os.path.join(path, f'{market}_{h}_30Y.csv')

            # Read the entire CSV file into a DataFrame, but only the rows with date in 'dates_dt'
            df = pd.read_csv(d_path, parse_dates=[0], index_col=0)
            df.index = df.index.astype(str).str.split(" ").str[0]
            df.index = pd.to_datetime(df.index)
            df = df[df.index.isin(dates_dt)]

            # Transpose the DataFrame so that the dates become columns and the fields become rows
            df_T = df.transpose()

            # Select only the rows for the fields we're interested in, which are the ones with indices 1 to 5
            df_selected = df_T.iloc[0:5]

            # Convert the selected part of the DataFrame to a numpy array and assign it to the tensor
            X[:, idx, :] = torch.from_numpy(df_selected.to_numpy())

        return X

    def _create_graphs(self, dates: List[str], desti: str, comlist: List[str], market: str, root: str, window: int, next_day: str):
        dates.append(next_day)

        # Wrap the range function with tqdm to create a progress bar
        for i in tqdm(range(len(dates) - window)):
            # Construct the filename for the current graph
            directory_path = os.path.join(desti, f'{market}_{self.dataset_type}_{self.start}_{self.end}_{window}')
            filename = os.path.join(directory_path, f'graph_{i}.pt')

            # If the file already exists, skip to the next iteration
            if os.path.exists(filename):
                print(f"Graph {i + 1}/{len(dates) - window} already exists, skipping...")
                continue

            print(f'Generating graph {i + 1}/{len(dates) - window}...')

            # Generate labels
            box = dates[i:i + window + 1]
            X = self.node_feature_matrix(box, comlist, market, root)
            C = torch.zeros(X.shape[1])
            for j in range(C.shape[0]):
                if X[3, j, -1] - X[3, j, -2] > 0:
                    C[j] = 1

            # Slice the desired data and do normalization on raw data
            X = X[:, :, :-1]
            for i in range(X.shape[0]):
                # Adding 1 before taking log to avoid log(0)
                X[i] = torch.Tensor(np.log1p(X[i].numpy()))

            # Obtain adjacency tensor
            A = torch.zeros((X.shape[0], X.shape[1], X.shape[1]))
            for j in range(A.shape[0]):
                A[j] = self.adjacency_matrix(X[j])

            # Save the X, A, C tensors
            os.makedirs(directory_path, exist_ok=True)
            torch.save({'X': X, 'A': A, 'Y': C}, filename)

# ***Model Configuration***

In [ ]:
import torch
import torch.nn as nn


class MultiReDiffusion(torch.nn.Module):

    def __init__(self, input_dim, output_dim, num_relation):
        super(MultiReDiffusion, self).__init__()
        self.output = output_dim
        self.fc_layers = nn.ModuleList([nn.Linear(input_dim, output_dim) for _ in range(num_relation)])
        self.update_layer = torch.nn.Conv2d(num_relation, num_relation, kernel_size=1)
        self.activation1 = torch.nn.PReLU()
        self.activation0 = torch.nn.PReLU()
        self.num_relation = num_relation

    def forward(self, theta, t, a, x):
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        diffusions = torch.zeros(theta.shape[0], a.shape[1], self.output).to(device)

        for rel in range(theta.shape[0]):
            diffusion_mat = torch.zeros_like(a[rel])
            for step in range(theta.shape[-1]):
                diffusion_mat += theta[rel][step] * t[rel][step] * a[rel]

            diffusion_feat = torch.matmul(diffusion_mat, x[rel])
            diffusions[rel] = self.activation0(self.fc_layers[rel](diffusion_feat))

        latent_feat = self.activation1(self.update_layer(diffusions.unsqueeze(0)))
        latent_feat = latent_feat.reshape(self.num_relation, a.shape[1], -1)

        return latent_feat, diffusions


class ParallelRetention(torch.nn.Module):

    def __init__(self, time_dim, in_dim, inter_dim, out_dim):
        super(ParallelRetention, self).__init__()
        self.time_dim = time_dim
        self.in_dim = in_dim
        self.inter_dim = inter_dim
        self.out_dim = out_dim
        self.activation = torch.nn.PReLU()
        self.Q_layers = nn.Linear(self.in_dim, self.inter_dim)
        self.K_layers = nn.Linear(self.in_dim, self.inter_dim)
        self.V_layers = nn.Linear(self.in_dim, self.inter_dim)
        self.ret_feat = torch.nn.Linear(self.inter_dim, self.out_dim)

    def forward(self, x, d_gamma):
        num_node = x.shape[1]
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        d_gamma = d_gamma.to(device)
        x = x.view(self.time_dim, -1)

        inter_feat = self.Q_layers(x) @ self.K_layers(x).transpose(0, 1)
        x = (d_gamma * inter_feat) @ self.V_layers(x)
        x = self.activation(self.ret_feat(x))

        return x.view(num_node, -1)


class MGDPR(nn.Module):
    def __init__(self, diffusion, retention, ret_linear_1, ret_linear_2, post_pro,
                 layers, num_nodes, time_dim, num_relation, gamma, expansion_steps):
        super(MGDPR, self).__init__()

        self.layers = layers

        # Learnable parameters for multi-relational transitions:
        # T is used as a transition (or weighting) tensor.
        # Initialized using standard Xavier uniform initialization.
        self.T = nn.Parameter(torch.empty(layers, num_relation, expansion_steps, num_nodes, num_nodes))
        nn.init.xavier_uniform_(self.T)

        # Theta is used for weighting coefficients in the diffusion process.
        self.theta = nn.Parameter(torch.empty(layers, num_relation, expansion_steps))
        nn.init.xavier_uniform_(self.theta)

        # Create a lower triangular mask. Only positions with lower_tri != 0 (i.e., strictly lower triangular)
        # will be assigned a decay value computed as gamma ** -lower_tri.
        lower_tri = torch.tril(torch.ones(time_dim, time_dim), diagonal=-1)
        D_gamma_tensor = torch.where(lower_tri == 0, torch.tensor(0.0), gamma ** -lower_tri)
        # Register as a buffer so it moves with the model's device and is saved/loaded with state_dict.
        self.register_buffer('D_gamma', D_gamma_tensor)

        # Initialize Multi-relational Graph Diffusion layers.
        self.diffusion_layers = nn.ModuleList(
            [MultiReDiffusion(diffusion[i], diffusion[i + 1], num_relation)
             for i in range(len(diffusion) - 1)]
        )

        # Initialize Parallel Retention layers.
        self.retention_layers = nn.ModuleList(
            [ParallelRetention(time_dim, retention[3 * i], retention[3 * i + 1], retention[3 * i + 2])
             for i in range(len(retention) // 3)]
        )

        # Initialize decoupled transformation layers.
        self.ret_linear_1 = nn.ModuleList(
            [nn.Linear(ret_linear_1[2 * i], ret_linear_1[2 * i + 1])
             for i in range(len(ret_linear_1) // 2)]
        )
        self.ret_linear_2 = nn.ModuleList(
            [nn.Linear(ret_linear_2[2 * i], ret_linear_2[2 * i + 1])
             for i in range(len(ret_linear_2) // 2)]
        )

        # MLP layers for post-processing.
        self.mlp = nn.ModuleList(
            [nn.Linear(post_pro[i], post_pro[i + 1]) for i in range(len(post_pro) - 1)]
        )


    def forward(self, x, a):
        """
        x: input tensor (e.g., node features); expected shape should be (batch_size, num_nodes, feature_dim)
        a: adjacency (or relation) information for graph diffusion.
        """
        # Use the same device as the input x.
        device = x.device

        # Ensure h is on the proper device.
        h = x.to(device)

        # Information diffusion and graph representation learning
        for l in range(self.layers):
            # Multi-relational Graph Diffusion layer:
            # The diffusion layer returns updated h and an intermediate representation u.
            h, u = self.diffusion_layers[l](self.theta[l], self.T[l], a, h)

            # Ensure u is on the same device as D_gamma.
            u = u.to(device)

            # Parallel Retention layer:
            # The retention layer expects u and the decay matrix D_gamma.
            eta = self.retention_layers[l](u, self.D_gamma)

            # Decoupled representation transform:
            # For the first layer, combine the eta representation with a transformed version of the
            # original input.
            if l == 0:
                # Reshape x to (num_nodes, -1) so that it aligns with eta.
                x_reshaped = x.view(x.shape[1], -1)
                h_concat = torch.cat((eta, self.ret_linear_1[l](x_reshaped)), dim=1)
                h_prime = self.ret_linear_2[l](h_concat)
            else:
                h_concat = torch.cat((eta, self.ret_linear_1[l](h_prime)), dim=1)
                h_prime = self.ret_linear_2[l](h_concat)

        # Post-processing with MLP layers to generate final graph representation.
        for mlp_layer in self.mlp:
            h_prime = mlp_layer(h_prime)

        return h_prime


    def reset_parameters(self):
        """
        Reset learnable model parameters using appropriate initialization methods.
        Note that D_gamma is not learnable and is registered as a buffer.
        """
        # Reinitialize T and theta with Xavier uniform initialization.
        nn.init.xavier_uniform_(self.T)
        nn.init.xavier_uniform_(self.theta)

        # Optionally, you could also reset the parameters of the submodules.
        for module in self.modules():
            if hasattr(module, 'reset_parameters') and module not in [self]:
                module.reset_parameters()


# ***Train/Validate/Test***

In [ ]:
import csv as csv
import torch.nn.functional as F
import torch.distributions
from sklearn.metrics import matthews_corrcoef, f1_score

# Configure the device for running the model on GPU or CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Configure the default variables // # these can be tuned // # examples
sedate = ['2013-01-01', '2014-12-31']  # these can be tuned
val_sedate = ['2015-01-01', '2015-06-30'] # these can be tuned
test_sedate = ['2015-07-01', '2017-12-31'] # these can be tuned
market = ['NASDAQ', 'NYSE', 'SSE'] # can be changed
dataset_type = ['Train', 'Validation', 'Test']
com_path = ['/content/drive/MyDrive/Raw_Data/Stock_Markets/NYSE_NASDAQ/NASDAQ.csv',
            '/content/drive/MyDrive/Raw_Data/Stock_Markets/NYSE_NASDAQ/NYSE.csv',
            '/content/drive/MyDrive/Raw_Data/Stock_Markets/NYSE_NASDAQ/NYSE_missing.csv']
des = '/content/drive/MyDrive/Raw_Data/Stock_Markets/NYSE_NASDAQ/raw_stock_data/stocks_indicators/data'
directory = "/content/drive/MyDrive/Raw_Data/Stock_Markets/NYSE_NASDAQ/raw_stock_data/stocks_indicators/data/google_finance"

NASDAQ_com_list = []
NYSE_com_list = []
NYSE_missing_list = []
com_list = [NASDAQ_com_list, NYSE_com_list, NYSE_missing_list]
for idx, path in enumerate(com_path):
    with open(path) as f:
        file = csv.reader(f)
        for line in file:
            com_list[idx].append(line[0])  # append first element of line if each line is a list
NYSE_com_list = [com for com in NYSE_com_list if com not in NYSE_missing_list]

# Generate datasets
train_dataset = MyDataset(directory, des, market[0], NASDAQ_com_list, sedate[0], sedate[1], 19, dataset_type[0])
validation_dataset = MyDataset(directory, des, market[0], NASDAQ_com_list, sedate[0], sedate[1], 19, dataset_type[0])
test_dataset = MyDataset(directory, des, market[0], NASDAQ_com_list, sedate[0], sedate[1], 19, dataset_type[0])

# Define model (these can be tuned)
n = len(NASDAQ_com_list) # number of companies in NASDAQ

d_layers, num_nodes, time_steps, num_relation, gamma, diffusion_steps = 6, n, 21, 5, 2.5e-4, 7

diffusion_layers = [time_steps, 3 * time_steps, 4 * time_steps, 5 * time_steps, 5 * time_steps, 6 * time_steps, 5 * time_steps]

retention_layers = [num_relation*3*n, num_relation*5*n, num_relation*4*n,
                    num_relation*4*n, num_relation*5*n, num_relation*5*n,
                    num_relation*5*n, num_relation*5*n, num_relation*5*n,
                    num_relation*5*n, num_relation*5*n, num_relation*5*n,
                    num_relation*6*n, num_relation*5*n, num_relation*5*n,
                    num_relation*5*n, num_relation*5*n, num_relation*5*n]


ret_linear_layers_1 = [time_steps * num_relation, time_steps * num_relation,
                     time_steps * num_relation * 5, time_steps * num_relation,
                     time_steps * num_relation * 6, time_steps * num_relation,
                     time_steps * num_relation * 6, time_steps * num_relation,
                     time_steps * num_relation * 6, time_steps * num_relation,
                     time_steps * num_relation * 6, time_steps * num_relation]


ret_linear_layers_2 = [time_steps * num_relation * 5, time_steps * num_relation * 5,
                     time_steps * num_relation * 6, time_steps * num_relation * 6,
                     time_steps * num_relation * 6, time_steps * num_relation * 6,
                     time_steps * num_relation * 6, time_steps * num_relation * 6,
                     time_steps * num_relation * 6, time_steps * num_relation * 6,
                     time_steps * num_relation * 6, time_steps * num_relation * 6]

mlp_layers = [num_relation * 5 * time_steps + time_steps * num_relation, 128, 2]

# Define model
model = MGDPR(diffusion_layers, retention_layers, ret_linear_layers_1, ret_linear_layers_2, mlp_layers, d_layers,
              num_nodes, time_steps, num_relation, gamma, diffusion_steps)

# Pass model and datasets to GPU
model = model.to(device)

# Define optimizer and objective function


def theta_regularizer(theta):
    row_sums = torch.sum(theta.to(device), dim=-1)
    ones = torch.ones_like(row_sums)
    return torch.sum(torch.abs(row_sums - ones))


#def D_gamma_regularizer(D_gamma):
    #upper_tri = torch.triu(D_gamma, diagonal=1)
    #return torch.sum(torch.abs(upper_tri))

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)

# Define training process & validation process & testing process
epochs = 10000
model.reset_parameters()

# Training and validating
for epoch in range(epochs):
    model.train()
    objective_total = 0
    correct = 0
    total = 0

    for sample in train_dataset: # Recommand to update every sample, full batch training can be time-consuming
        X = sample['X'].to(device)  # node feature tensor
        A = sample['A'].to(device)  # adjacency tensor
        C = sample['Y'].long()
        C = C.to(device)  # label vector

        optimizer.zero_grad()
        out = model(X, A)
        objective = F.cross_entropy(out, C) #+ theta_regularizer(model.theta) regularization may resultvery slow learning process, optional usage.
        objective.backward()
        optimizer.step()
        objective_total += objective.item()

    # If performance progress of the model is required
        out = model(X, A).argmax(dim=1)
        correct += int((out == C).sum()).item()
        total += C.shape[0]
        if epoch % 1 == 0:
          print(f"Epoch {epoch}: loss={objective_total:.4f}, acc={correct / total:.4f}")

# Validation
model.eval()
acc = 0
f1 = 0
mcc = 0

for idx, sample in enumerate(validation_dataset):
    X = sample['X']  # node feature tensor
    A = sample['A']  # adjacency tensor
    C = sample['Y']  # label vector
    out = model(X, A).argmax(dim=1)

    acc += int((out == C).sum())
    f1 += f1_score(C, out.cpu().numpy())
    mcc += matthews_corrcoef(C, out.cpu().numpy())

print(acc / (len(validation_dataset) * C.shape[0]))
print(f1 / len(validation_dataset))
print(mcc / len(validation_dataset))

# Test
acc = 0
f1 = 0
mcc = 0

for idx, sample in enumerate(test_dataset):
    X = sample['X']  # node feature tensor
    A = sample['A']  # adjacency tensor
    C = sample['Y']  # label vector
    out = model(X, A).argmax(dim=1)

    acc += int((out == C).sum())
    f1 += f1_score(C, out.cpu().numpy())
    mcc += matthews_corrcoef(C, out.cpu().numpy())

print(acc / (len(test_dataset) * C.shape[0]))
print(f1 / len(test_dataset))
print(mcc / len(test_dataset))

# save model to the directory
if int(input('save model? (1/0)?')) == 1:
    torch.save(model, dir_path() + 'your_dataset_name/model')